Class Name: Data Science for Linguistics. 

Project Name: Analysis the distribution of Chinese Dialects among small datasets  

Authors:     Jiarong Tang      Weiting Wang  

Descripition: Use the Chinese word2Vec Embedding, string distance, network tree and language models to show the distribution of Chinese dialects. Because the original data is big, including 203 words and 19 cities, it is not impossible to show all words in all cities' distribtions. So we just choose several words among a few famous cities, such as Beijing, Guangzhou and so on.

In [99]:
import pandas as pd
from gensim.models import KeyedVectors


In [87]:
data_df = pd.read_csv('forms.csv')
data_df

,ID,Local_ID,Language_ID,Parameter_ID,Value,Form,Segments,Comment,Source,Cognacy,Loan,Graphemes,Profile,Prosody,Morpheme_Glosses,Partial_Cognacy,Chinese_Characters
0,Beijing-91_vomit-1,NaN,Beijing,91_vomit,tʰu⁵¹,tʰu⁵¹,tʰ u ⁵¹,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t,spit/吐,1,吐
1,Haerbin-91_vomit-1,NaN,Haerbin,91_vomit,tʰu⁵³,tʰu⁵³,tʰ u ⁵³,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t,spit/吐,1,吐
2,Jinan-91_vomit-1,NaN,Jinan,91_vomit,tʰu³¹,tʰu³¹,tʰ u ³¹,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t,spit/吐,1,吐
3,Rongcheng-91_vomit-1,NaN,Rongcheng,91_vomit,ou²¹³⁻³⁵ tʰu²¹⁴,ou²¹³⁻³⁵ tʰu²¹⁴,ou ²¹³ + tʰ u ²¹⁴,copulative synonyme,Liu2007,NaN,NaN,NaN,NaN,n t + i n t,nausea/嘔 spit/吐,2 1,嘔 吐
4,Taiyuan-91_vomit-1,NaN,Taiyuan,91_vomit,tʰu⁵³ lə⁰,tʰu⁵³ lə⁰,tʰ u ⁵³ + l ə ⁰,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t + i n t,nausea/嘔 _:PERFECTIVE/了,2 5,嘔 嘞
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4297,Guangzhou-90_woman-1,NaN,Guangzhou,90_woman,nøy²³ iɐn²¹⁻²³,nøy²³ iɐn²¹⁻²³,n øy ²³ + j ɐ n ²¹,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t + i n c t,female/女 _person/人,39 38,女 人
4298,Fuzhou-90_woman-1,NaN,Fuzhou,90_woman,i⁵⁵⁻⁵³ tsia³²,i⁵⁵⁻⁵³ tsia³²,i ⁵⁵ + ts j a ³²,NaN,Liu2007,NaN,NaN,NaN,NaN,n t + i m n t,woman/伊 sister/姐,824 715,伊 姐
4299,Fuzhou-90_woman-2,NaN,Fuzhou,90_woman,ny³²⁻⁵⁵ ɛ²¹²,ny³²⁻⁵⁵ ɛ²¹²,n y ³² + ɛ ²¹²,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t + n t,female/女 _world/界,39 825,女 界
4300,Fuzhou-90_woman-3,NaN,Fuzhou,90_woman,tsy⁵⁵ nøyŋ⁵³⁻⁵⁵ nøyŋ⁵³,tsy⁵⁵ nøyŋ⁵³⁻⁵⁵ nøyŋ⁵³,ts y ⁵⁵ + n øy ŋ ⁵³ + n øy ŋ ⁵³,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t + i n c t + i n c t,woman/諸 female/娘 _person/人,749 31 38,諸 娘 儂


In [88]:
# get the info of the dataFrame columns
data_df.columns

Index(['ID', 'Local_ID', 'Language_ID', 'Parameter_ID', 'Value', 'Form',
       'Segments', 'Comment', 'Source', 'Cognacy', 'Loan', 'Graphemes',
       'Profile', 'Prosody', 'Morpheme_Glosses', 'Partial_Cognacy',
       'Chinese_Characters'],
      dtype='object')

step 1 extract all the parameter_ID/ Language_ID to decide how many words we are inverstigating


In [89]:
# 19 cities  but there is variety among the same city 
cities = data_df['Language_ID'].unique()
cities

array(['Beijing', 'Haerbin', 'Jinan', 'Rongcheng', 'Taiyuan', 'XiAn',
       'Chengdu', 'Nanjing', 'Jixi', 'Suzhou', 'Wenzhou', 'Changsha',
       'Loudi', 'Nanchang', 'Meixian', 'Guilin', 'Guangzhou', 'Fuzhou',
       'Xiamen'], dtype=object)

In [ ]:
#  比如沿河城市对一个单词的相似性或者两个
#  沿着山脉的对一个单词的相似性或者两个
# 7 cities
#  'Suzhou',   # 东部'Guangzhou', # 南部'Chengdu',   # 西部'Beijing',   # 北部'Haerbin',   # 东北部（补充选择）
# 'Changsha',  # 中部（补充选择）'XiAn'  

In [90]:
# 203 words 
words = data_df['Parameter_ID'].unique()
words

array(['91_vomit', '92_fear', '93_skin', '94_float', '95_smooth',
       '96_wife', '97_all', '98_hot', '99_person', '100_throw', '1_white',
       '2_back', '3_nose', '4_ice', '5_neck', '6_thin', '7_not', '8_rub',
       '9_grass', '10_long', '101_flesh', '102_if', '103_breasts',
       '104_three', '105_woods', '106_kill', '107_sand', '108_mountain',
       '109_burn', '110_few', '111_tongue', '112_snake', '113_rope',
       '114_louse', '115_wet', '116_what', '117_stone', '118_hand',
       '119_tree', '120_bark', '121a_countnoun', '121b_countverb',
       '122_who', '123_water', '124_fruit', '125_sleep', '126_suck',
       '127_say', '128_split', '129_die', '130_four', '131_he',
       '132_they', '133_sun', '134_lie', '135_day', '136_sky', '137_hear',
       '138_head', '139_hair', '140_earth', '141_spit', '142_push',
       '143_leg', '144_dig', '145_play', '146_night', '147_tail',
       '148_smell', '149_i', '150_we', '151_five', '152_fog', '153_knee',
       '154_wash', '155_t

In [91]:
len(words)

203

In [92]:
# method to get infos of the same parameter_ID
def get_characters(para,info1,info2):
    return (data_df[data_df['Parameter_ID']== para][info1],data_df[data_df['Parameter_ID']== para][info2])


In [93]:
a,b = get_characters('91_vomit','ID','Chinese_Characters')

In [94]:
a

0       Beijing-91_vomit-1
1       Haerbin-91_vomit-1
2         Jinan-91_vomit-1
3     Rongcheng-91_vomit-1
4       Taiyuan-91_vomit-1
5          XiAn-91_vomit-1
6       Chengdu-91_vomit-1
7       Chengdu-91_vomit-2
8       Chengdu-91_vomit-3
9       Nanjing-91_vomit-1
10      Nanjing-91_vomit-2
11         Jixi-91_vomit-1
12       Suzhou-91_vomit-1
13      Wenzhou-91_vomit-1
14     Changsha-91_vomit-1
15        Loudi-91_vomit-1
16        Loudi-91_vomit-2
17     Nanchang-91_vomit-1
18     Nanchang-91_vomit-2
19      Meixian-91_vomit-1
20       Guilin-91_vomit-1
21    Guangzhou-91_vomit-1
22       Fuzhou-91_vomit-1
23       Xiamen-91_vomit-1
Name: ID, dtype: object

In [95]:
type(b)

pandas.core.series.Series

In [96]:
# for loop the words and get the infos of character of different cities
char_dict = {}
for word in words:
    char_dict[word]=get_characters(word,'ID','Chinese_Characters')
         

In [97]:
char_dict

{'91_vomit': (0       Beijing-91_vomit-1
  1       Haerbin-91_vomit-1
  2         Jinan-91_vomit-1
  3     Rongcheng-91_vomit-1
  4       Taiyuan-91_vomit-1
  5          XiAn-91_vomit-1
  6       Chengdu-91_vomit-1
  7       Chengdu-91_vomit-2
  8       Chengdu-91_vomit-3
  9       Nanjing-91_vomit-1
  10      Nanjing-91_vomit-2
  11         Jixi-91_vomit-1
  12       Suzhou-91_vomit-1
  13      Wenzhou-91_vomit-1
  14     Changsha-91_vomit-1
  15        Loudi-91_vomit-1
  16        Loudi-91_vomit-2
  17     Nanchang-91_vomit-1
  18     Nanchang-91_vomit-2
  19      Meixian-91_vomit-1
  20       Guilin-91_vomit-1
  21    Guangzhou-91_vomit-1
  22       Fuzhou-91_vomit-1
  23       Xiamen-91_vomit-1
  Name: ID, dtype: object,
  0         吐
  1         吐
  2         吐
  3       嘔 吐
  4       嘔 嘞
  5       嘔 吐
  6     發 吐 了
  7       吐 了
  8       嘔 了
  9         吐
  10      嘔 吐
  11      㽹 惡
  12        嘔
  13        吐
  14        吐
  15        囗
  16      帶 囗
  17        嘔
  18        吐

Q1. Do cities that are geographically close have similar characteristics in the same Parameter_ID?

In [ ]:
# char dict {word, (ID-Series, Char-Seires)}
# compare multiple words? this means make a similarity matrix
# matrix row and column is the city names but 
# the content(value) is based on the computation of characters' embedding similarity
# matrix fill in 
# use agorithm ??? pandas 
# show it 




https://github.com/IBM/MAX-Chinese-Phonetic-Similarity-Estimator
# IBM tool

https://ai.tencent.com/ailab/nlp/en/embedding.html  

tencent 

In [103]:
pwd()

'/Users/weiting/Desktop/2024summer semester/data linguistics'

In [106]:
cd tencent-ailab-embedding-zh-d100-v0.2.0

/Users/weiting/Desktop/2024summer semester/data linguistics/tencent-ailab-embedding-zh-d100-v0.2.0


In [108]:
# import word2vec Model
wv_from_text = KeyedVectors.load_word2vec_format('tencent-ailab-embedding-zh-d100-v0.2.0.txt', binary=False)
    

In [114]:
def get_similarity(word1, word2, model):
    word1 = word1.replace(' ', '')
    word2 = word2.replace(' ', '')
    if word1 in wv_from_text.key_to_index and word2 in wv_from_text.key_to_index:
        similarity = wv_from_text.similarity(word1, word2)
        print(f"The similarity between '{word1}' and '{word2}' is: {similarity}")
    else:
        print(f"One or both words are not in the model's vocabulary.")
           
        
    

In [116]:
get_similarity('嘔 吐','吐',wv_from_text)

The similarity between '嘔吐' and '吐' is: 0.5806554555892944


Use word2vec embeddings to test the similarities

Q2 under the same word, test the value/form/segements difference among different areas

But with what method to calculate the phonetics difference? Use value /Form or Segments

Q3 does the similarity of embedding has the positive coefficient with values simialrity

Q4 use the network to show the tree 

Q5 Model to show the dialect distribution 

Q6 N-Gram Analysis 

Q7 the same city such as Chengdu ,Nnajing and so on has varieties , what is the reason for this variety 

Q8 the words without any change, what is the reason ?

step3 Raise some questions related with the dialetcs' geography and explore it 